In [7]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from model_cnn_lstm import build_cnn_lstm_model
from utils.dataset_preparation import extract_frames


In [ ]:
# Convert all videos into frame sequences (.npy)
extract_frames("dataset/train/Accident", "dataset/preprocessed/accident")
extract_frames("dataset/train/Normal", "dataset/preprocessed/normal")


In [8]:
from tensorflow.keras.utils import Sequence
import numpy as np
import os

class VideoDataGenerator(Sequence):
    def __init__(self, accident_dir, normal_dir, batch_size):
        self.accident_files = [os.path.join(accident_dir, f) for f in os.listdir(accident_dir)]
        self.normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir)]
        self.files = self.accident_files + self.normal_files
        self.labels = [1]*len(self.accident_files) + [0]*len(self.normal_files)
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.files) / self.batch_size))

    def __getitem__(self, idx):
        batch_files = self.files[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_labels = self.labels[idx*self.batch_size : (idx+1)*self.batch_size]

        X, y = [], []
        for file, label in zip(batch_files, batch_labels):
            data = np.load(file)
            X.append(data)
            y.append(label)

        X = np.array(X)
        y = np.eye(2)[y]  # One-hot encode
        return X, y


In [ ]:
# Create data generator
train_gen = VideoDataGenerator("dataset/preprocessed/accident", "dataset/preprocessed/normal", batch_size=4)

# Build model
model = build_cnn_lstm_model()
model.summary()

# Train model
history = model.fit(
    train_gen,
    epochs=25,
    verbose=1
)

# Save model
model.save("model/accident_detector_cnn_lstm.h5")
print("✅ Model trained and saved successfully!")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_1 (TimeDistributed) │ (None, 16, 2048)            │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │       2,360,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,981,186 (99.11 MB)

 Trainable params: 2,393,474 (9.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
